### Tomographic Reconstruction (IMOD)

Reconstruction was performed with IMOD BatchRunTomo.

Reconstruction algorithm: WBP

For more information, please see https://bio3d.colorado.edu/imod/doc/directives.html

In [ ]:
def get_tomogram_filenames():
    # Get IMOD mdout
    try:
        imod_mdout = f"{os.getcwd()}/{proj_name}_recon_mdout.yaml"
    except FileNotFoundError:
        print(f"{proj_name}_recon_mdout.yaml not found, cannot find tomogram filenames.")


    with open(imod_mdout, "r") as f:
        output = yaml.load(f.read(), Loader=yaml.FullLoader)
        tomogram_filenames = output["recon_output"].values()

    return tomogram_filenames

tomogram_filenames = get_tomogram_filenames()

In [ ]:
def get_central_slices(img_data):
    # Find central slices
    z_central = int(img_data.shape[0]/2)
    y_central = int(img_data.shape[1]/2)
    x_central = int(img_data.shape[2]/2)

    # xy, yz, xz image data
    xy_data = img_data[z_central, :, :]
    yz_data = img_data[:, y_central, :]
    xz_data = img_data[:, :, x_central]

    return [xy_data, yz_data, xz_data]


def show_tomogram(tomo_fname):
    """ tomo_fname can be 3-D .mrc, 3-D .tiff, or stack of 2-D tiffs """

    # Import image data
    if tomo_fname.endswith(".mrc"):
        with mrcfile.open(tomo_fname) as mrc:
            img_data = mrc.data
    
    elif os.path.isdir(tomo_fname):
        volume = []
        for xy_slice in os.listdir(tomo_fname):
            volume.append(tifffile.imread(f"{tomo_fname}/{xy_slice}").data)
        img_data = np.stack(volume)
    
    elif tomo_fname.endswith(".tif"):
        img_data = tifffile.imread(tomo_fname)
    
    else:
        raise ValueError("Tomogram cannot be read. Please use a 3D tiff, mrc or stack of 2D Tiffs")
        
    xy_data, yz_data, xz_data = get_central_slices(img_data)
    # Show central slices in xy, yz, xz
    plt.figure(figsize=(10,3.5))
    plt.suptitle(os.path.basename(tomo_fname))
    titles = ["x-y", "y-z", "x-z"]

    for i, data in enumerate([xy_data, yz_data, xz_data]):
        plt.subplot(1,3,i+1)
        plt.imshow(
            data,
            cmap="Greys_r",
            vmin=np.percentile(data.flatten(), 10),
            vmax=np.percentile(data.flatten(), 90)
        )
        plt.title(titles[i])
    
    plt.tight_layout()

In [ ]:
# Show thumbnails of the tomograms

for tomogram in tomogram_filenames:
    show_tomogram(tomogram)